<a href="https://colab.research.google.com/github/AbhishekGarg03/Projects/blob/main/mubert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/spaces/Mubert/Text-to-Music

Cloning into 'Text-to-Music'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 34 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [ ]:
cd Text-to-Music

/content/Text-to-Music


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 68 kB 3.5 MB/s 
     |████████████████████████████████| 54 kB 1.3 MB/s 
     |████████████████████████████████| 80 kB 4.8 MB/s 
     |████████████████████████████████| 5.3 MB 59.4 MB/s 
     |████████████████████████████████| 1.3 MB 69.7 MB/s 
     |████████████████████████████████| 163 kB 65.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=90d0011a1a5aa605107918f109ddcd2994fe0906447693939b8247b5b64dc567
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 7.7 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 212 kB 46.9 MB/s 
     |████████████████████████████████| 2.3 MB 52.9 MB/s 
     |████████████████████████████████| 272 kB 72.0 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 106 kB 73.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 4.0 MB 34.0 MB/s 
     |████████████████████████████████| 593 kB 86.3 MB/s 
     |████████████████████████████████| 856 kB 86.3 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=cd6c350e49c8f4aeb70f8ca35b9ba2024f01aee38dde9158ca21778345569930
  Stored i

In [ ]:
import time

import gradio as gr
from sentence_transformers import SentenceTransformer

import httpx
import json

from utils import get_tags_for_prompts, get_mubert_tags_embeddings, get_pat

minilm = SentenceTransformer('all-MiniLM-L6-v2')
mubert_tags_embeddings = get_mubert_tags_embeddings(minilm)


def get_track_by_tags(tags, pat, duration, maxit=20, loop=False):
    if loop:
        mode = "loop"
    else:
        mode = "track"
    r = httpx.post('https://api-b2b.mubert.com/v2/RecordTrackTTM',
                   json={
                       "method": "RecordTrackTTM",
                       "params": {
                           "pat": pat,
                           "duration": duration,
                           "tags": tags,
                           "mode": mode
                       }
                   })

    rdata = json.loads(r.text)
    assert rdata['status'] == 1, rdata['error']['text']
    trackurl = rdata['data']['tasks'][0]['download_link']

    print('Generating track ', end='')
    for i in range(maxit):
        r = httpx.get(trackurl)
        if r.status_code == 200:
            return trackurl
        time.sleep(1)


def generate_track_by_prompt(email, prompt, duration, loop=False):
    try:
        pat = get_pat(email)
        _, tags = get_tags_for_prompts(minilm, mubert_tags_embeddings, [prompt, ])[0]
        return get_track_by_tags(tags, pat, int(duration), loop=loop), "Success"
    except Exception as e:
        return None, str(e)


block = gr.Blocks()

with block:
    gr.HTML(
        """
            <div style="text-align: center; max-width: 700px; margin: 0 auto;">
              <div
                style="
                  display: inline-flex;
                  align-items: center;
                  gap: 0.8rem;
                  font-size: 1.75rem;
                "
              >
                <h1 style="font-weight: 900; margin-bottom: 7px;">
                  Mubert
                </h1>
              </div>
              <p style="margin-bottom: 10px; font-size: 94%">
                All music is generated by Mubert API – <a href="www.mubert.com/" style="text-decoration: underline;" target="_blank">www.mubert.com/</a>
              </p>
            </div>
        """
    )
    with gr.Group():
        with gr.Box():
            email = gr.Textbox(label="email")
            prompt = gr.Textbox(label="prompt")
            duration = gr.Slider(label="duration (seconds)", value=30)
            out = gr.Audio()
            result_msg = gr.Text(label="Result message")
            btn = gr.Button("Submit").style(full_width=True)
        
        btn.click(fn=generate_track_by_prompt, inputs=[email, prompt, gr.Slider(label="duration (seconds)", value=30)], outputs=[out,result_msg]) 
        gr.HTML('''
        <div class="footer" style="text-align: center; max-width: 700px; margin: 0 auto;">
                    <p>Demo by <a href="https://huggingface.co/Mubert" style="text-decoration: underline;" target="_blank">Mubert</a>
                    </p>
        </div>
        ''')

block.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://2a792d51569a9638.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fe4ef9429d0>,
 'http://127.0.0.1:7861/',
 'https://2a792d51569a9638.gradio.app')